In [1]:
from pprint import pprint
from llama_index.core import Settings
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.llms.groq import Groq
import nest_asyncio
nest_asyncio.apply()
api_key="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"
llm = Groq(model="llama3-8b-8192",
    api_key=api_key,
    temperature=0.5
)
response = llm.complete("hi")
response.text

/home/adminn/Desktop/Repositories/database-chat-assistant/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


"Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

bacis llama single document agent experiment

In [2]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["LordOfTheRings.pdf"]).load_data()
len(documents)

1347

In [3]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)
pprint(nodes[12].metadata, indent=4)

{   'creation_date': '2024-05-16',
    'file_name': 'LordOfTheRings.pdf',
    'file_path': 'LordOfTheRings.pdf',
    'file_size': 13674248,
    'file_type': 'application/pdf',
    'last_modified_date': '2024-05-14',
    'page_label': '13'}


In [4]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embediing_model = HuggingFaceEmbedding(model_name='thenlper/gte-base')
Settings.llm = llm
Settings.embed_model=embediing_model

Two indexes that llm will choose to use when quering the vDB

In [10]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import SummaryIndex, VectorStoreIndex
from app.utils.connections import chromadb_connection
summary_index = SummaryIndex(nodes, show_progress=True)
summary_collection = chromadb_connection("sum_idx")
vector_index = VectorStoreIndex(nodes, show_progress=True)
vector_collection = chromadb_connection("sum_idx")
chroma_vec_store = ChromaVectorStore(chroma_collection=vector_collection)
chroma_doc_store = ChromaVectorStore(chroma_collection=summary_collection)

Generating embeddings: 100%|██████████| 1347/1347 [1:46:21<00:00,  4.74s/it]


Initializing query engine to perform search

In [17]:
from llama_index.core.tools import QueryEngineTool

engine_summ = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True
)
engine_vect = vector_index.as_query_engine()

summary_tool = QueryEngineTool.from_defaults(
    query_engine=engine_summ,
    description="Used when queries are related to Summary of entire document."
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=engine_vect,
    description="Used when queries require specific context instead of entire document."
)
engine_summ

Define a Router using the query engine tools created using indexes

In [31]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        # summary_tool,
        vector_tool
    ],
    verbose=True
)

In [38]:
response = query_engine.query("what actually is durins bane")
print(str(response))
print(len(response.source_nodes))

Selecting query engine 0: The question is asking about the context of Durin's Bane, which is a specific concept in J.R.R. Tolkien's Middle-earth legendarium. The summary suggests that this concept requires specific context instead of the entire document, which is consistent with the idea that Durin's Bane is a specific entity that requires context to understand..
Durin's Bane is not explicitly mentioned in the provided context. However, it can be inferred that Durin's Bane is likely a powerful and ancient evil that has been awakened or is about to be awakened, given the descriptions of the world being fair in Durin's Day and the mention of mighty kings and cities that have passed away.
2


Funtion tool calling basics

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import FunctionTool

vector_index = VectorStoreIndex(nodes, show_progress=True)
query_engine = vector_index.as_query_engine(similarity_top_k=2)
pprint(nodes[2].get_content(metadata_mode="all"), indent=4)